# Summary

> Obsidian App Summary Dashboard

In [ ]:
#| default_exp home.summary

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from memexplatform_obsidian.datastore import FileStore, NoteStore, DBStore, iter_files
from memexplatform_obsidian.commons import config, ExtensionTypes
import pandas as pd

In [ ]:
#| notest
fstore = FileStore(config)
dirs = fstore.subdirs; dirs
for f in iter_files(dirs): 
    infos = f.lstat(); print(infos)
    break

os.stat_result(st_mode=33188, st_ino=171505886, st_dev=16777230, st_nlink=1, st_uid=1803992684, st_gid=826136866, st_size=57683, st_atime=1756674677, st_mtime=1756562895, st_ctime=1756562895)


In [ ]:
#| export
def get_df(config, fldrs = ['pages', 'journals'], extensions = ExtensionTypes.TEXTLIKE_EXTS):
    store =  DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    flds = ",".join(f"'{e}'" for e in fldrs)
    exts = ",".join(f"'{e.lstrip('.')}'"for e in  extensions); exts
    a = list(store.db.query(f"""
        select title, url, created_time, modified_time from node 
        where is_folder=0 
        and parent in ({flds})
        and ext in ({exts})
        """)); a
    # ); a

    if a:
        df = pd.DataFrame.from_records(a); df
        df['created_time'] = pd.to_datetime(df['created_time'],  unit='s', utc=True).dt.date
        df['modified_time'] = pd.to_datetime(df['modified_time'], unit='s', utc=True).dt.date
        return df
    else: return None

In [ ]:
#| export
def chart_tags():
    store =  DBStore(config) if config.ENABLE_DB_MODE else FileStore(config)
    query = f"""
    select distinct n.title, n.url, p.value as tag
    from node n
    join properties p on n.lockey = p.lockey
    where p.type = 'TagLink'
    and n.parent in ('pages', 'journals', 'Clippings')
    """
    a = list(store.db.query(query)); a
    df = pd.DataFrame.from_records(a); df
    # df.groupby('tag')['title'].count().sort_values(ascending=False)
    # Aggregate counts
    tag_counts = (
        df.groupby("tag")["title"]
        .count()
        .sort_values(ascending=False)
    )

    series_data = tag_counts.values.tolist()
    categories = tag_counts.index.tolist()

    return ApexChart(
        opts={
            "series": [
                {"name": "Notes", "data": series_data}
            ],
            "chart": {
                "type": "bar",
                "height": 400
            },
            "plotOptions": {
                "bar": {
                    "borderRadius": 4,
                    "horizontal": True
                }
            },
            "dataLabels": {"enabled": False},
            "xaxis": {
                "categories": categories,
                "title": {"text": "Count"}
            },
            "yaxis": {
                "title": {"text": "Tag"}
            },
            "legend": {"show": False}
        },
        cls="max-w-3xl max-h-lg"
    )

In [ ]:
chart_tags()

```html
<div class="max-w-3xl max-h-lg">
<uk-chart><script type='application/json'>{"series": [{"name": "Notes", "data": [13, 12, 9, 6, 4, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]}], "chart": {"type": "bar", "height": 400}, "plotOptions": {"bar": {"borderRadius": 4, "horizontal": true}}, "dataLabels": {"enabled": false}, "xaxis": {"categories": ["#clippings", "#inbox", "#Vault", "#Project", "#Person", "#excalidraw", "#Pillar", "#Thought", "#banker", "#conservative", "#democrat", "#endregion", "#female", "#impression", "#liberal", "#region"], "title": {"text": "Count"}}, "yaxis": {"title": {"text": "Tag"}}, "legend": {"show": false}}</script></uk-chart></div>

```

In [ ]:

#| notest
df = get_df(config)
df_created = df.groupby('created_time')['title'].count()
df_modified = df.groupby('modified_time')['title'].count()


created_series = [
    {"x": d.strftime("%Y-%m-%d"), "y": int(v)}
    for d, v in df_created.items()
]; created_series


modified_series = [
    {"x": d.strftime("%Y-%m-%d"), "y": int(v)}
    for d, v in df_modified.items()
]; modified_series

[{'x': '2025-08-30', 'y': 2},
 {'x': '2025-08-31', 'y': 10},
 {'x': '2025-09-01', 'y': 4},
 {'x': '2025-09-02', 'y': 7},
 {'x': '2025-09-04', 'y': 7},
 {'x': '2025-09-06', 'y': 4},
 {'x': '2025-09-07', 'y': 1},
 {'x': '2025-09-08', 'y': 2},
 {'x': '2025-09-09', 'y': 1},
 {'x': '2025-09-10', 'y': 3},
 {'x': '2025-09-12', 'y': 6},
 {'x': '2025-09-13', 'y': 2},
 {'x': '2025-09-14', 'y': 3},
 {'x': '2025-09-18', 'y': 1},
 {'x': '2025-09-21', 'y': 4},
 {'x': '2025-09-22', 'y': 7},
 {'x': '2025-09-23', 'y': 1},
 {'x': '2025-09-25', 'y': 3},
 {'x': '2025-09-27', 'y': 2},
 {'x': '2025-09-28', 'y': 2}]

In [ ]:
#| export
def InfoCard(title, value, change): 
    return Card(H3(value), footer=P(change, cls=TextPresets.muted_sm), header = H4(title))

In [ ]:
#| notest
df = get_df(config, fldrs=['pages', 'journals'], extensions=ExtensionTypes.TEXTLIKE_EXTS)
s = df.groupby('created_time')['title'].count()


In [ ]:
#| export
def get_infocard(df, header='Notes'):
    if df is not None:
        count = df.count()['title']
        s = df.groupby('created_time')['title'].count()
        m = f"+{s[s.index.max()]} last on  {s.index.max()}";m
    else:
        count = 0
        s = 0
        m = "No additional info"
    return InfoCard(header, count, m)

In [ ]:
#| export
def chart_timeseries(df):
    # Convert dataframe into Apex format
    df_created = df.groupby('created_time')['title'].count()
    df_modified = df.groupby('modified_time')['title'].count()
    created_series = [
        {"x": d.strftime("%Y-%m-%d"), "y": int(v)}
        for d, v in df_created.items()
    ]; created_series


    modified_series = [
        {"x": d.strftime("%Y-%m-%d"), "y": int(v)}
        for d, v in df_modified.items()
    ]; modified_series

    return ApexChart(
        opts={
            "chart": {
                "type": "line",
                "zoom": {"enabled": True},
                "toolbar": {"show": True}
            },
            "stroke": {"curve": "smooth"},
            "series": [
                {"name": "Created",  "data": created_series},
                {"name": "Modified", "data": modified_series}
            ],
            "xaxis": {
                "type": "datetime",
                "title": {"text": "Date"}
            },
            "yaxis": {
                "title": {"text": "Count"}
            },
            "markers": {"size": 4},
            "tooltip": {"shared": True, "intersect": False},
            "legend": {"position": "top"}
        },
        cls="max-w-3xl max-h-lg"
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()